# Import Packages

In [5]:
import os
import pickle as pkl
import sys
sys.path.append('/Users/kana/Library/Mobile Documents/com~apple~CloudDocs/Codes/GWOT_colorprefrencequalia')
import numpy as np
import pandas as pd
import sklearn
import csv
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import MDS
import seaborn as sns
import ot
import plotly.graph_objs as go
import plotly.express as px
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Our own utility functions
import utilityFunctions

# Define variables (used across the code)

In [6]:
# Define unique colors
unique_colours = np.array(['#d2b700', '#db8b08', '#c7512c', '#c13547', '#a03663', '#753a7a', '#4b488e', '#005692', '#006a8b', '#007b75', '#008a52', '#9aa400'])
colour_index = {colour: idx for idx, colour in enumerate(unique_colours)}
matrix_size = len(unique_colours)

### configuration
n_eps = 15 # number of epsilon values tried
eps_range = [0.04, 5] # the range of epsilon searched
epsilons = np.logspace(np.log10(eps_range[0]), np.log10(eps_range[1]), n_eps) # epsilon values

# Load Data

## Raw Data - List of all subject data

In [ ]:
# Function for getting data

data_dir = 'raw_data/kana_colourpreferencequalia-master/data'

# TODO add parameter which allows for filtering of data based on specified column-value pairs
def getData(data_dir, target_data):
    # Extract data from datafiles in provided directory
    # INPUTS:
    #   data_dir: string, directory holding csv data files
    #   target_data: string, target column name in data files which has the data you want to get
    # OUTPUTS:
    #   pMatrices:
    #   pIds:
    #   pFiles:


    pFiles = [] # stores source datafile for each participant, pFiles[pID] gives the file for participant pID
    pCounter = 0 

    pMatrices = [] # stores data matrix for each participant

    for filename in os.listdir(data_dir):
        if filename.endswith(".csv"):
            filepath = os.path.join('raw_data/kana_colourpreferencequalia-master/data', filename)

            # This is another participant
            pFiles.append(filename)
            pCounter = pCounter + 1
            
            # Load the CSV file
            df = pd.read_csv(filepath)

            # Filter for rows where 'response_type' is 'similarity'
            df_similarity = df[df['response_type'] == 'similarity']

            # Extract columns
            colour1 = df_similarity['colour1']
            colour2 = df_similarity['colour2']
            target_preference = df_similarity['response']

            # Create and fill the matrix
            # NOTE double pass trials will overwrite the first trials (is this what we want?)
            matrix_size = len(colour_index)
            matrix = np.zeros((matrix_size, matrix_size))
            for c1, c2, tp in zip(colour1, colour2, target_preference):
                I = colour_index[c1]
                j = colour_index[c2]
                matrix[j, I] = tp

            # Store the matrix
            pMatrices.append(matrix)

    pIds = range(0, pCounter)

    return pMatrices, pIds, pFiles

In [ ]:
# Load datafiles for each participant and extract similarity matrices

pFiles = [] # stores source datafile for each participant, pFiles[pID] gives the file for participant pID
pCounter = 0 

pMatrices = [] # stores data matrix for each participant

for filename in os.listdir('raw_data/kana_colourpreferencequalia-master/data'):
    if filename.endswith(".csv"):
        filepath = os.path.join('raw_data/kana_colourpreferencequalia-master/data', filename)

        # This is another participant
        pFiles.append(filename)
        pCounter = pCounter + 1
        
        # Load the CSV file
        df = pd.read_csv(filepath)

        # Filter for rows where 'response_type' is 'similarity'
        df_similarity = df[df['response_type'] == 'similarity']

        # Extract columns
        colour1 = df_similarity['colour1']
        colour2 = df_similarity['colour2']
        target_preference = df_similarity['response']

        # Create and fill the matrix
        # NOTE double pass trials will overwrite the first trials (is this what we want?)
        matrix_size = len(colour_index)
        matrix = np.zeros((matrix_size, matrix_size))
        for c1, c2, tp in zip(colour1, colour2, target_preference):
            I = colour_index[c1]
            j = colour_index[c2]
            matrix[j, I] = tp

        # Store the matrix
        pMatrices.append(matrix)

pIds = range(0, pCounter)

In [ ]:
# Show matrices
utilityFunctions.show_heatmaps(0, 7, matrices=pMatrices, titles=["subject" + str(p) for p in pIds], cbar_label="similarity", color_labels=unique_colours)

for f in range(0, len(pFiles)):
    print("subject" + str(f) + ' ' + pFiles[f])


In [ ]:
for filename in os.listdir('raw_data/kana_colourpreferencequalia-master/data'):
    if filename.endswith(".csv"):
        filepath = os.path.join('raw_data/kana_colourpreferencequalia-master/data', filename)
        
        # Load the CSV file
        df = pd.read_csv(filepath)

        # Filter for rows where 'response_type' is 'similarity'
        df_similarity = df[df['response_type'] == 'similarity']

        # Extract columns
        colour1 = df_similarity['colour1']
        colour2 = df_similarity['colour2']
        target_preference = df_similarity['response']

        # Create and fill the matrix
        matrix_size = len(colour_index)
        matrix = np.zeros((matrix_size, matrix_size))
        for c1, c2, tp in zip(colour1, colour2, target_preference):
            I = colour_index[c1]
            j = colour_index[c2]
            matrix[j, I] = tp

        # Plot the heatmap for this file
        show_heatmap(matrix, filename, cbar_label="similarity", color_labels=unique_colours)

In [ ]:
pCorrs = []

folder_path = 'raw_data/kana_colourpreferencequalia-master/data'

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        
        df = pd.read_csv(filepath, usecols=['response_type', 'response', 'colour1', 'colour2', 'trials.thisIndex', 'practice_trial'])

        # Omit practice trials
        df = df[df['practice_trial'] != 1]  

        df_similarity = df[df['response_type'] == 'similarity']

        # Group by 'trials.thisIndex'
        grouped = df_similarity.groupby('trials.thisIndex')['response'].apply(list).reset_index()

        # Filter for pairs with exactly two trials
        repeated_pairs = grouped[grouped['response'].apply(len) == 2]

        # Check if there are any repeated pairs
        if len(repeated_pairs) > 0:
            # Flatten the responses for repeated pairs
            responses_trial1 = np.array(repeated_pairs['response'].apply(lambda x: x[0]).tolist())
            responses_trial2 = np.array(repeated_pairs['response'].apply(lambda x: x[1]).tolist())

            # Calculate overall correlation for repeated color pairs
            overall_correlation = pearsonr(responses_trial1, responses_trial2)[0]

            pCorrs.append(overall_correlation)

            print(f"Overall Pearson Correlation for {filename}: {overall_correlation}")
        else:
            print(f"No repeated pairs found in {filename}")
    
    plt.hist(pCorrs, color='blue')
    plt.xlabel("r")
    plt.ylabel("N")
    plt.title('double pass - similarity')